Test du projet de class trajets indirect

In [1]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\Martin\git\Outils\Outils\Martin_Perso')
import traitement_od as t
import fiona
import pandas as pd
import numpy as np
import altair as alt
import datetime as dt

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 85: invalid continuation byte

Exception ignored in: 'fiona._env.log_error'
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 85: invalid continuation byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 85: invalid continuation byte

Exception ignored in: 'fiona._env.log_error'
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 85: invalid continuation byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 86: invalid continuation byte

...

Exception ignored in: 'fiona._env.log_error'
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 85: invalid continuation byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 88: invalid continuation byte

Exception ignored in: 'fiona._env.log_error'
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 88: invalid continuation byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 88: invalid continuation byte

Exception ignored in: 'fiona._env.log_error'
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 88: invalid continuation byte


In [2]:
df_journee=t.ouvrir_fichier_lapi('2019-01-29 00:00:00','2019-01-30 23:59:00')

In [3]:
toto=t.trajet_indirect(df_journee,'2019-01-29 08:00:00', 60, 16, [19,4,5])

In [4]:
toto.dico_traj_directs['trajet1'].df_tps_parcours_pl_final

KeyError: 'trajet1'

In [34]:
toto.dico_traj_directs

{'trajet0': <traitement_od.df_tps_parcours at 0x1e18eda79e8>,
 'trajet1': <traitement_od.df_tps_parcours at 0x1e18eda7a58>}

In [35]:
toto.df_trajet_indirect()

,immat,date_cam_1,date_cam_2,cam_1,cam_2,tps_parcours
0,bf9dfd50a4a693085e3080156f11c25314f96f03,2019-01-29 08:00:56,2019-01-29 08:37:18,19,5,00:36:22
1,cf4f7ddbb3a6d9006e029777df712a53c2d76992,2019-01-29 08:01:07,2019-01-29 08:37:04,19,5,00:35:57
2,183bc8585ac5a2e26b6d2ada6ce6891235a36b92,2019-01-29 08:01:16,2019-01-29 08:37:20,19,5,00:36:04
3,dcf2ffa7638b1ae7d0281cbb94d4c7cf0a0912f7,2019-01-29 08:02:34,2019-01-29 08:37:42,19,5,00:35:08
4,54581cfa0b0f0c1dbe24fd60ca7efcc07439232f,2019-01-29 08:02:19,2019-01-29 08:38:07,19,5,00:35:48
5,31c6bdae383e5c4f25e803d55dc539d6c40e6105,2019-01-29 08:03:06,2019-01-29 08:38:16,19,5,00:35:10
6,77728d75d8ee8a1e27c95ca4077faddbf9dcdd93,2019-01-29 08:03:55,2019-01-29 08:38:48,19,5,00:34:53
7,59148b1dc67f7b2ca10635cd4fabe903b1fd0b63,2019-01-29 08:04:14,2019-01-29 08:39:21,19,5,00:35:07
8,3de49b107ce9b3f1add8f9314add9f5a2cadbe50,2019-01-29 08:05:07,2019-01-29 08:39:49,19,5,00:34:42
9,2a88c4e3e59236321a2098ccc9dd4b47f2eed478,2019-01-29 08:07:23,2019-01-29 08:42:27,19,5,00:35:04


In [12]:
len(dico)

3

In [30]:
for a, val_dico in enumerate(dico):
    if a<=len(dico)-2:
        variab='traj'+str(a)
        variab2='traj'+str(a+1)
        if 'result' in locals() : 
            result=result+dico[variab2]
            print('ensuite',variab,variab2,result  )
        else : 
            result=dico[variab]+dico[variab2]
            print('initial',variab,variab2,result  )

initial traj0 traj1 6
ensuite traj1 traj2 14


In [31]:
result

14

In [32]:
del(result)